# Exploring Used Cars Data

## Import libraries

In [1]:
# Libraries for data analysis
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format',  '{:,.2f}'.format)

# Libraries for visualizations
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Ignore any warnings
import warnings
warnings.filterwarnings('ignore')

## Read in the dataset

In [2]:
df = pd.read_csv('USA_cars_datasets.csv', index_col=0)

# See first five rows
df.head()

,price,brand,model,year,title_status,mileage,color,vin,lot,state,country,condition
0,6300,toyota,cruiser,2008,clean vehicle,"274,117.00",black,jtezu11f88k007763,159348797,new jersey,usa,10 days left
1,2899,ford,se,2011,clean vehicle,"190,552.00",silver,2fmdk3gc4bbb02217,166951262,tennessee,usa,6 days left
2,5350,dodge,mpv,2018,clean vehicle,"39,590.00",silver,3c4pdcgg5jt346413,167655728,georgia,usa,2 days left
3,25000,ford,door,2014,clean vehicle,"64,146.00",blue,1ftfw1et4efc23745,167753855,virginia,usa,22 hours left
4,27700,chevrolet,1500,2018,clean vehicle,"6,654.00",red,3gcpcrec2jg473991,167763266,florida,usa,22 hours left


In [3]:
# Information about the dataset
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2499 entries, 0 to 2498
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   price         2499 non-null   int64  
 1   brand         2499 non-null   object 
 2   model         2499 non-null   object 
 3   year          2499 non-null   int64  
 4   title_status  2499 non-null   object 
 5   mileage       2499 non-null   float64
 6   color         2499 non-null   object 
 7   vin           2499 non-null   object 
 8   lot           2499 non-null   int64  
 9   state         2499 non-null   object 
 10  country       2499 non-null   object 
 11  condition     2499 non-null   object 
dtypes: float64(1), int64(3), object(8)
memory usage: 253.8+ KB


- The dataset has **2499 rows** and **12 columns**.

## Let's explore the numerical data first.

In [4]:
df.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
price,"2,499.00","18,767.67","12,116.09",0.00,"10,200.00","16,900.00","25,555.50","84,900.00"
year,"2,499.00","2,016.71",3.44,"1,973.00","2,016.00","2,018.00","2,019.00","2,020.00"
mileage,"2,499.00","52,298.69","59,705.52",0.00,"21,466.50","35,365.00","63,472.50","1,017,936.00"
lot,"2,499.00","167,691,389.06","203,877.21","159,348,797.00","167,625,331.00","167,745,058.00","167,779,772.00","167,805,500.00"


**Observations**
- The **average price** of a car in the dataset is **$18,768**. The **oldest car** in the dataset is from **1973** and the **newest** car is from **2020**. 
- There are a few outliers that will need to be dealt with. The minimum price and mileage cannot be 0 since we are dealing with used cars, and the car with the highest mileage has over 1 million miles on it. It doesn't seem feasible for a car to last for over 1 million miles.

## Next let's look at the categorical data.

In [5]:
df.describe(include='O').transpose()

,count,unique,top,freq
brand,2499,28,ford,1235
model,2499,127,door,651
title_status,2499,2,clean vehicle,2336
color,2499,49,white,707
vin,2499,2495,1gnevhkw8jj148388,2
state,2499,44,pennsylvania,299
country,2499,2,usa,2492
condition,2499,47,2 days left,832


**Observations**
- Ford is the company that has the most cars featured in this dataset with 1235. 
- The most popular colour for a car is white. 
- Pennsylvania is the most represented state.

## Data Cleaning

In [6]:
# Drop unnecessary columns
df = df.drop(['vin', 'lot'], axis=1)

df.head()

,price,brand,model,year,title_status,mileage,color,state,country,condition
0,6300,toyota,cruiser,2008,clean vehicle,"274,117.00",black,new jersey,usa,10 days left
1,2899,ford,se,2011,clean vehicle,"190,552.00",silver,tennessee,usa,6 days left
2,5350,dodge,mpv,2018,clean vehicle,"39,590.00",silver,georgia,usa,2 days left
3,25000,ford,door,2014,clean vehicle,"64,146.00",blue,virginia,usa,22 hours left
4,27700,chevrolet,1500,2018,clean vehicle,"6,654.00",red,florida,usa,22 hours left


In [7]:
# Check for null values
df.isna().sum()

price           0
brand           0
model           0
year            0
title_status    0
mileage         0
color           0
state           0
country         0
condition       0
dtype: int64

In [8]:
# Check for duplicates
df.duplicated().sum()

0

### Detecting Numeric Outliers

In [9]:
df[['price', 'mileage']].describe()

,price,mileage
count,"2,499.00","2,499.00"
mean,"18,767.67","52,298.69"
std,"12,116.09","59,705.52"
min,0.00,0.00
25%,"10,200.00","21,466.50"
50%,"16,900.00","35,365.00"
75%,"25,555.50","63,472.50"
max,"84,900.00","1,017,936.00"


**Observations**

##### Price
- The minimum price of 0 is unusual and could affect the analysis. 
- The maximum price is $84,900 which does not seem far-fetched.

##### Mileage
- The minimum mileage for a car is 0 and is unexpected since we are dealing with used cars.
- The maximum mileage for a car is over 1,000,000 miles which does not seem feasible.

#### Cleaning the Price Column

In [10]:
Q1 = df['price'].quantile(0.25)
Q3 = df['price'].quantile(0.75)
IQR = Q3 - Q1

# Getting the quartiles in a price format
Q1_dollars = "$ {:.2f}".format(Q1)
Q3_dollars = "$ {:.2f}".format(Q3)
IQR_dollars = "$ {:.2f}".format(IQR)


print("The 25th percentile is " +  str(Q1_dollars))
print("The 75th percentile is " +  str(Q3_dollars))
print("The Interquartile Range is " + str(IQR_dollars))

The 25th percentile is $ 10200.00
The 75th percentile is $ 25555.50
The Interquartile Range is $ 15355.50


In [11]:
# Upper and Lower Fence
lower_fence = Q1 - (1.5 * IQR)
upper_fence = Q3 + (1.5 * IQR)

print('Lower Fence: ', lower_fence)
print('Upper Fence: ', upper_fence)

Lower Fence:  -12833.25
Upper Fence:  48588.75


In [101]:
# Plot the price distribution
fig = px.histogram(df, x="price", nbins=30)

fig.update_xaxes(title='Price', showgrid=False, showline=False, linewidth=2, linecolor='black', ticks='outside')
fig.update_yaxes(title='Count', showgrid=True, showline=False, linewidth=1, linecolor='black')
fig.update_layout(font_size = 12, font_family='Arial')
fig.show()

In [13]:
# Check how many values of 0 we have in the price column
df['price'].value_counts().head()

0        43
16500    26
13900    21
15500    19
15000    19
Name: price, dtype: int64

In [14]:
# Only keep cars which have prices between $1 and the upper fence of $48,589
df = df[df['price'].between(1, upper_fence)]

df['price'].describe()

count    2,392.00
mean    18,116.04
std     10,432.18
min         25.00
25%     10,400.00
50%     16,800.00
75%     25,000.00
max     48,500.00
Name: price, dtype: float64

- We lose 107 cars from the dataset which had a price listing of less than 1 dollar and more than 48,589 dollars. 
- The standard deviation is $10,432, meaning the variation in prices of the cars is quite large.

In [15]:
# Plot the new price distribution after cleaning
fig = px.histogram(df, x="price", nbins=20)

fig.update_xaxes(title='Price', showgrid=False, showline=False, linewidth=2, linecolor='black', ticks='outside')
fig.update_yaxes(title='Count', showgrid=False, showline=False, linewidth=1, linecolor='black')
fig.update_layout(font_size = 12, font_family='Arial')
fig.show()

- The most common price range for a car to fall in is between 15,000 dollars and 20,000 dollars. There are 461 cars in this price range.
- The second most common price range is between 10,000 dollars and 15,000 dollars.
- The price distribution looks much more like a normal distribution now.

###  Cleaning the Mileage Column

In [16]:
Q1_mileage = df['mileage'].quantile(0.25)
Q3_mileage = df['mileage'].quantile(0.75)
IQR_mileage = Q3_mileage - Q1_mileage

print("The 25th percentile is " +  str(Q1_mileage))
print("The 75th percentile is " +  str(Q3_mileage))
print("The Interquartile Range is " + str(IQR_mileage))

The 25th percentile is 22098.75
The 75th percentile is 61687.25
The Interquartile Range is 39588.5


In [17]:
lower_fence_mileage = Q1_mileage - (1.5 * IQR_mileage)
upper_fence_mileage = Q3_mileage + (1.5 * IQR_mileage)

print('Lower Fence Mileage: ', lower_fence_mileage)
print('Upper Fence Mileage: ', upper_fence_mileage)

Lower Fence Mileage:  -37284.0
Upper Fence Mileage:  121070.0


In [18]:
# Let's drop cars with mileage of $0 and and also cars with mileage greater than the upper fence of $121,070
df = df[df['mileage'].between(1, upper_fence_mileage)]

df['mileage'].describe()

count     2,213.00
mean     40,590.19
std      27,101.66
min           1.00
25%      20,849.00
50%      33,909.00
75%      51,347.00
max     120,667.00
Name: mileage, dtype: float64

- We lose an additional 179 cars by using this method of removing outliers from the mileage column
- It brings down the maximum mileage to 120,667 and reduces the standard deviation to 27,101.66
- The good news is that we still have over 2250 cars to analyze

In [19]:
fig = px.histogram(df, x="mileage", nbins=15)

fig.update_xaxes(title='Mileage', showgrid=False, showline=False, linewidth=2, linecolor='black', ticks='outside')
fig.update_yaxes(title='Count', showgrid=False, showline=False, linewidth=1, linecolor='black')
fig.update_layout(font_size = 12, font_family='Arial')
fig.show()

- The mileage range which a majority of the vehicles fall into is between 30000 and 40000 miles. 489 cars fall in this range.
- The next most popular mileage range is 10000 to 20000 miles. This range has 401 cars in it.
- The mileage column has a long tail, meaning there are a not non-significant number of cars in each 10,000 mile range above 60,000 miles up to 120,000 miles.

### Question: How does mileage on a car affect its price?

In [20]:
# Let's bin the mileage column into 12 equal bins of 10,000 miles each
df['mileage_bins'] = pd.cut(df['mileage'], bins=np.arange(0, 130000, 10000))

df['mileage_bins'].value_counts()

(30000, 40000]      489
(10000, 20000]      401
(20000, 30000]      369
(40000, 50000]      256
(50000, 60000]      133
(0, 10000]          128
(60000, 70000]       87
(80000, 90000]       83
(70000, 80000]       81
(90000, 100000]      70
(100000, 110000]     66
(110000, 120000]     48
Name: mileage_bins, dtype: int64

In [21]:
# View the new column
df.head()

,price,brand,model,year,title_status,mileage,color,state,country,condition,mileage_bins
2,5350,dodge,mpv,2018,clean vehicle,"39,590.00",silver,georgia,usa,2 days left,"(30000, 40000]"
3,25000,ford,door,2014,clean vehicle,"64,146.00",blue,virginia,usa,22 hours left,"(60000, 70000]"
4,27700,chevrolet,1500,2018,clean vehicle,"6,654.00",red,florida,usa,22 hours left,"(0, 10000]"
5,5700,dodge,mpv,2018,clean vehicle,"45,561.00",white,texas,usa,2 days left,"(40000, 50000]"
7,13350,gmc,door,2017,clean vehicle,"23,525.00",gray,california,usa,20 hours left,"(20000, 30000]"


In [22]:
# Average price of a car for each mileage bin
price_by_mileage = df.groupby('mileage_bins')['price'].agg({'mean', 'count'})

price_by_mileage.rename(columns={"mean": 'average_price'}, inplace=True)

price_by_mileage.index = price_by_mileage.index.astype(str)

price_by_mileage

,count,average_price
mileage_bins,,
"(0, 10000]",128,"24,206.66"
"(10000, 20000]",401,"24,349.72"
"(20000, 30000]",369,"21,888.54"
"(30000, 40000]",489,"18,215.37"
"(40000, 50000]",256,"17,703.27"
"(50000, 60000]",133,"17,248.10"
"(60000, 70000]",87,"15,251.92"
"(70000, 80000]",81,"14,067.46"
"(80000, 90000]",83,"12,609.75"


In [23]:
fig = px.bar(price_by_mileage, y='average_price', text='count', template='plotly_white', title='Price by Mileage Bin')

fig.update_xaxes(title='Mileage_Bins', showline=True, linewidth=2, linecolor='black', ticks='outside')
fig.update_yaxes(title='Price', showgrid=True, showline=True, linewidth=1, linecolor='black')
fig.update_layout(font_size = 12, font_family='Arial')
fig.show()

#### Observations:
- For the most part, as the mileage increases, cars become less expensive.
- The only exception to this is that cars with mileage between 10,000 and 20,000 are priced slightly higher than those between 0 and 10000 miles.

#### Price to mileage ratio

In [24]:
df['pm_ratio'] = df['price'] / df['mileage']

df.sort_values(by='pm_ratio', ascending=False).head()

,price,brand,model,year,title_status,mileage,color,state,country,condition,mileage_bins,pm_ratio
2085,31000,ford,door,2017,clean vehicle,1.00,black,illinois,usa,2 days left,"(0, 10000]","31,000.00"
1479,26000,ford,f-150,2018,clean vehicle,1.00,black,illinois,usa,2 days left,"(0, 10000]","26,000.00"
342,6680,heartland,country,2011,clean vehicle,1.00,gray,pennsylvania,usa,7 days left,"(0, 10000]","6,680.00"
325,4050,heartland,pioneer,2018,clean vehicle,1.00,gray,pennsylvania,usa,7 days left,"(0, 10000]","4,050.00"
2232,4000,nissan,sentra,2019,clean vehicle,1.00,black,arizona,usa,2 days left,"(0, 10000]","4,000.00"


In [25]:
# Only inclue vehicles with at least 1000 miles on them because pm_ratio is skewed heavily to vehicles with low mileage
df2 = df[df['mileage'] >= 1000]

df2.sort_values(by='pm_ratio', ascending=False).head(10)

,price,brand,model,year,title_status,mileage,color,state,country,condition,mileage_bins,pm_ratio
1714,43900,ford,explorer,2020,clean vehicle,"2,089.00",no_color,south carolina,usa,21 hours left,"(0, 10000]",21.01
966,19500,dodge,caravan,2019,clean vehicle,"1,091.00",gray,arizona,usa,20 hours left,"(0, 10000]",17.87
1269,35000,ford,mustang,2018,clean vehicle,"2,042.00",blue,illinois,usa,21 hours left,"(0, 10000]",17.14
1759,37500,ford,srw,2019,clean vehicle,"2,382.00",silver,nevada,usa,5 days left,"(0, 10000]",15.74
1486,37000,ford,f-150,2019,clean vehicle,"2,384.00",gray,pennsylvania,usa,3 days left,"(0, 10000]",15.52
1388,17000,ford,fusion,2020,clean vehicle,"1,117.00",gray,pennsylvania,usa,2 days left,"(0, 10000]",15.22
1253,30000,ford,ranger,2019,clean vehicle,"2,590.00",gray,wisconsin,usa,21 hours left,"(0, 10000]",11.58
94,39800,dodge,challenger,2015,clean vehicle,"3,641.00",white,pennsylvania,usa,2 days left,"(0, 10000]",10.93
646,46000,chevrolet,corvette,2016,clean vehicle,"4,445.00",black,illinois,usa,2 days left,"(0, 10000]",10.35
1676,23400,ford,fusion,2020,clean vehicle,"2,452.00",gray,michigan,usa,21 hours left,"(0, 10000]",9.54


In [26]:
df2['pm_ratio'].describe()

count   2,194.00
mean        0.90
std         1.35
min         0.00
25%         0.25
50%         0.52
75%         1.11
max        21.01
Name: pm_ratio, dtype: float64

- We lose 120 cars by filtering for cars with at least 1000 miles on them.
- The highest pm_ratio is 21.01 and the lowest pm_ratio is 0

In [27]:
max_ratio = df2['pm_ratio'].max()
min_ratio = df2['pm_ratio'].min()

ratio_diff = max_ratio - min_ratio

ratio_diff

21.014602092956697

In [28]:
# Create a new "value earned" column
df2['value_earned'] = ((10 * (df['pm_ratio'] - min_ratio)) / ratio_diff)

# Top 10 most "valuable" cars
df2.sort_values(by='value_earned', ascending=False).head(10)

,price,brand,model,year,title_status,mileage,color,state,country,condition,mileage_bins,pm_ratio,value_earned
1714,43900,ford,explorer,2020,clean vehicle,"2,089.00",no_color,south carolina,usa,21 hours left,"(0, 10000]",21.01,10.00
966,19500,dodge,caravan,2019,clean vehicle,"1,091.00",gray,arizona,usa,20 hours left,"(0, 10000]",17.87,8.51
1269,35000,ford,mustang,2018,clean vehicle,"2,042.00",blue,illinois,usa,21 hours left,"(0, 10000]",17.14,8.16
1759,37500,ford,srw,2019,clean vehicle,"2,382.00",silver,nevada,usa,5 days left,"(0, 10000]",15.74,7.49
1486,37000,ford,f-150,2019,clean vehicle,"2,384.00",gray,pennsylvania,usa,3 days left,"(0, 10000]",15.52,7.39
1388,17000,ford,fusion,2020,clean vehicle,"1,117.00",gray,pennsylvania,usa,2 days left,"(0, 10000]",15.22,7.24
1253,30000,ford,ranger,2019,clean vehicle,"2,590.00",gray,wisconsin,usa,21 hours left,"(0, 10000]",11.58,5.51
94,39800,dodge,challenger,2015,clean vehicle,"3,641.00",white,pennsylvania,usa,2 days left,"(0, 10000]",10.93,5.20
646,46000,chevrolet,corvette,2016,clean vehicle,"4,445.00",black,illinois,usa,2 days left,"(0, 10000]",10.35,4.92
1676,23400,ford,fusion,2020,clean vehicle,"2,452.00",gray,michigan,usa,21 hours left,"(0, 10000]",9.54,4.54


In [106]:
# Filter for cars that are priced at $10,000 or less because that is my budget
df3 = df2[df2['price'] <= 10000]

# Only include 'clean vehicles'
df3 = df3[df3['title_status'] == 'clean vehicle']

# Top 10 most "valuable" cars priced at $10,000 or less
df3.sort_values(by='value_earned', ascending=False).head(10)

,price,brand,model,year,title_status,mileage,color,state,country,condition,mileage_bins,pm_ratio,value_earned
252,6530,jeep,mpv,2019,clean vehicle,"10,185.00",black,georgia,usa,17 hours left,"(10000, 20000]",0.64,0.30
148,4160,dodge,mpv,2018,clean vehicle,"6,743.00",gray,florida,usa,16 hours left,"(0, 10000]",0.62,0.29
1771,10000,ford,escape,2019,clean vehicle,"19,993.00",silver,illinois,usa,19 hours left,"(10000, 20000]",0.50,0.24
1945,7800,nissan,versa,2019,clean vehicle,"15,832.00",black,arizona,usa,2 days left,"(10000, 20000]",0.49,0.23
2181,9500,nissan,sentra,2019,clean vehicle,"19,646.00",black,california,usa,1 days left,"(10000, 20000]",0.48,0.23
292,9270,gmc,mpv,2017,clean vehicle,"19,490.00",gray,virginia,usa,17 hours left,"(10000, 20000]",0.48,0.23
249,7340,ford,doors,2019,clean vehicle,"15,854.00",gray,florida,usa,17 hours left,"(10000, 20000]",0.46,0.22
256,7400,ford,doors,2019,clean vehicle,"16,482.00",gray,florida,usa,17 hours left,"(10000, 20000]",0.45,0.21
109,9200,ford,door,2017,clean vehicle,"21,770.00",black,texas,usa,2 days left,"(20000, 30000]",0.42,0.20
98,8700,ford,focus,2018,clean vehicle,"21,405.00",white,south carolina,usa,21 hours left,"(20000, 30000]",0.41,0.19


- The **Jeep MPV** (Multi-Purpose Vehicle) is the most **"valuable"** car in this dataset priced at **$10,000 or less**.

### Year Column

In [132]:
cars_by_year = df.groupby('year')['model'].count().reset_index(name='count')

# Only include cars from 2011 or later (since their are so few cars for the years before then)
cars_by_year = cars_by_model_year[cars_by_model_year['year'] >= 2011]

cars_by_year

,year,count
9,2011,15
10,2012,32
11,2013,56
12,2014,75
13,2015,179
14,2016,195
15,2017,353
16,2018,388
17,2019,857
18,2020,45


### Question: What is the average price of a car by year for cars manufactured in 2011 or later?

In [150]:
# Average price of a car by year
price_by_year = df.groupby('year')['price'].agg({'mean', 'count'})

# Rename 'mean' to 'average_price'
price_by_year.rename(columns={"mean": 'average_price'}, inplace=True)

# Filter for 2011 and later
price_by_year = price_by_year[price_by_year.index >= 2011]

price_by_year

,count,average_price
year,,
2011,15,"8,274.33"
2012,32,"9,707.44"
2013,56,"9,693.30"
2014,75,"12,921.76"
2015,179,"15,758.92"
2016,195,"17,424.85"
2017,353,"17,406.84"
2018,388,"18,391.87"
2019,857,"22,744.35"


In [173]:
mileage_by_year = pd.DataFrame(df.groupby('year')['mileage'].mean())

# Filter for 2011 and later
mileage_by_year = mileage_by_year[mileage_by_year.index >= 2011]

mileage_by_year = pd.DataFrame(mileage_by_year['mileage'].round(decimals=0))

mileage_by_year

,mileage
year,
2011,"84,025.00"
2012,"84,567.00"
2013,"91,462.00"
2014,"73,883.00"
2015,"63,722.00"
2016,"56,833.00"
2017,"46,880.00"
2018,"34,357.00"
2019,"24,358.00"


In [186]:
price_by_year['text'] = price_by_year["count"].map(str) + ' cars, ' + mileage_by_year['mileage'].map(str) + ' miles'

price_by_year

,count,average_price,text
year,,,
2011,15,"8,274.33","15 cars, 84025.0 miles"
2012,32,"9,707.44","32 cars, 84567.0 miles"
2013,56,"9,693.30","56 cars, 91462.0 miles"
2014,75,"12,921.76","75 cars, 73883.0 miles"
2015,179,"15,758.92","179 cars, 63722.0 miles"
2016,195,"17,424.85","195 cars, 56833.0 miles"
2017,353,"17,406.84","353 cars, 46880.0 miles"
2018,388,"18,391.87","388 cars, 34357.0 miles"
2019,857,"22,744.35","857 cars, 24358.0 miles"


In [200]:
# Plot average price of a car by year
fig = px.bar(price_by_year, x=price_by_year.index, y='average_price', text=price_by_year['text'], template='plotly_white', 
             title='Average Price of a Car by Year')

fig.update_xaxes(type='category', title='Year', showgrid=False, showline=False, linewidth=2, linecolor='black', ticks='outside')
fig.update_yaxes(title='Price', showgrid=False, showline=False, linewidth=1, linecolor='black')
fig.update_layout(font_size = 12, font_family='Arial')
fig.update_traces(textangle=0, textfont_size=14)
fig.show()

- As expected, the newer the car is, the more expensive on average it is.
- **2019** is the most represented year with **857 cars**. The **average price** for a car from 2019 is **22,745 dollars**.
- For a budget conscious consumer, getting a car from **2018** as opposed to **2019** could **save you** more than **$4000 on average**.

### Categorical Column Exploration

In [30]:
# Resetting the index after the data cleaning
df2 = df2.reset_index(drop=True)

# Using df2 as it filters for cars with at least 1000 miles on them
df2.head()

,price,brand,model,year,title_status,mileage,color,state,country,condition,mileage_bins,pm_ratio,value_earned
0,5350,dodge,mpv,2018,clean vehicle,"39,590.00",silver,georgia,usa,2 days left,"(30000, 40000]",0.14,0.06
1,25000,ford,door,2014,clean vehicle,"64,146.00",blue,virginia,usa,22 hours left,"(60000, 70000]",0.39,0.19
2,27700,chevrolet,1500,2018,clean vehicle,"6,654.00",red,florida,usa,22 hours left,"(0, 10000]",4.16,1.98
3,5700,dodge,mpv,2018,clean vehicle,"45,561.00",white,texas,usa,2 days left,"(40000, 50000]",0.13,0.06
4,13350,gmc,door,2017,clean vehicle,"23,525.00",gray,california,usa,20 hours left,"(20000, 30000]",0.57,0.27


In [31]:
# Let's get the top 5 number of occurences for each column which is a 'object' column
for i in df2.select_dtypes(include=['object']).columns:
    display(pd.DataFrame(df2[i].value_counts().head()))

,brand
ford,1055
dodge,404
nissan,299
chevrolet,255
gmc,35


,model
door,490
f-150,211
doors,145
caravan,101
mpv,86


,title_status
clean vehicle,2141
salvage insurance,53


,color
white,618
black,457
gray,361
silver,278
red,164


,state
pennsylvania,269
florida,220
california,172
texas,171
michigan,157


,country
usa,2187
canada,7


,condition
2 days left,741
21 hours left,447
3 days left,117
14 hours left,105
1 days left,84


## Column by Column Exploration

### Brand column

In [32]:
print("There are " + str(df['brand'].nunique()) + " car brands represented in the dataset.")
print('---'*25)
print("The unique brands are: " + str(df['brand'].unique()))

There are 25 car brands represented in the dataset.
---------------------------------------------------------------------------
The unique brands are: ['dodge' 'ford' 'chevrolet' 'gmc' 'kia' 'buick' 'infiniti' 'mercedes-benz'
 'jeep' 'chrysler' 'bmw' 'cadillac' 'hyundai' 'heartland' 'jaguar' 'acura'
 'honda' 'lincoln' 'nissan' 'audi' 'land' 'maserati' 'lexus' 'ram' 'mazda']


#### There are 25 brands represented in this dataset. The only car brand I am not familiar with is 'heartland'. Let's explore the brand column some more.

In [33]:
# Top 10 Most popular brands
brand_df = pd.DataFrame(df['brand'].value_counts().head(10))

# Reset the index of the new dataframe
brand_df.reset_index(inplace=True)

# Rename the columns
brand_df.rename(columns={"index": 'brand', "brand": 'count'}, inplace=True)

brand_df

,brand,count
0,ford,1065
1,dodge,404
2,nissan,303
3,chevrolet,256
4,gmc,35
5,jeep,28
6,bmw,14
7,chrysler,14
8,hyundai,13
9,infiniti,12


In [34]:
fig = px.bar(brand_df, y='count', x='brand', template='plotly_white', title='Most Popular Car Brands', text='count')

fig.update_xaxes(title='Brand', showline=True, linewidth=2, linecolor='black', ticks='outside')
fig.update_yaxes(title='Count', showgrid=True, showline=True, linewidth=1, linecolor='black')
fig.update_layout(font_size = 12, font_family='Arial')
fig.show()

- Ford is by far the most represented brand with almost 1100 vehicles. Next are Dodge, Nissan and Chevrolet.
- The top 6 most represented brands are all American companies
- Every other company outside the top 4 has fewer than 50 cars represented. This tells me that I can get a solid understanding of the cars for these 4 brands, but I will be hard pressed to make conclusive statements about the cars of the other companies.

### Question: What is the average price of a car for each brand?

In [116]:
brand_price_df = df2.groupby('brand')['price'].agg({'mean','count'})

brand_price_df.reset_index(inplace=True)

In [117]:
# Only include the top 10 most represented brands
brand_price_df = brand_price_df.sort_values(by='count', ascending=False).head(10)

In [118]:
# Sort by the average price of each brand from highest to lowest
brand_price_df.sort_values(by='mean', ascending=False, inplace=True)

brand_price_df

,brand,count,mean
2,bmw,14,"23,860.71"
8,ford,1055,"21,539.30"
5,chevrolet,255,"20,400.51"
7,dodge,404,"18,214.84"
6,chrysler,14,"16,982.86"
12,infiniti,12,"13,098.33"
22,nissan,299,"12,445.67"
9,gmc,35,"12,217.43"
14,jeep,28,"11,706.79"
11,hyundai,13,"5,276.15"


In [36]:
# Plot the average price of the top 10 most represented brands
fig = px.bar(brand_price_df, y='mean', x='brand', template='plotly_white', 
             title='Average Price of Top 10 Most Represented Brands', text='count')

fig.update_xaxes(title='Brand', showline=True, linewidth=2, linecolor='black', ticks='outside')
fig.update_yaxes(title='Price', showgrid=True, showline=True, linewidth=1, linecolor='black')
fig.update_layout(font_size = 12, font_family='Arial')
fig.show()

- **BMW** has the highest average price of any car brand in the top 10 at **23,861 dollars**. This makes sense since it is a luxury car maker.
- **Ford** and **Chevrolet** are the next most expensive brands. This is surprising because these brands are considered more "affordable". I would expect these cars to have a similar average to **Nissan** which has an average price of **12,450 dollars**. In comparison, **Ford** cars average **21,540 dollars** and **Chevrolet** cars average **20,400 dollars**.

### Question: Which brands are the most valuable? Which brands are the least valuable?

- We have already created our "**value**" metric

In [37]:
brands_by_value = df2.groupby('brand')['value_earned'].agg({'mean','count'})

# Only include brands with at least 5 vehicles 
brands_by_value_5 = brands_by_value[brands_by_value['count'] >=5]

# Top 5 most valuable brands 
brands_by_value_5.sort_values(by='mean', ascending=False).head(5)

,count,mean
brand,,
chrysler,14,0.55
ford,1055,0.51
cadillac,9,0.48
buick,12,0.47
dodge,404,0.44


- The **Top 5 most valuable brands** are **Chrysler**, **Ford**, **Cadillac**, **Buick** and **Dodge**
- Although **Ford** has the **second highest average price** of any car company, they make up for it by being the **second most valuable brand**

In [38]:
# Top 5 least valuable brands
brands_by_value_5.sort_values(by='mean').head()

,count,mean
brand,,
hyundai,13,0.06
honda,9,0.07
kia,12,0.19
nissan,299,0.21
gmc,35,0.22


- The **Top 5 least valuable brands** are **Hyundai**, **Honda**, **Kia**, **Nissan** and **GMC**
- Although **Hyundai** has the **lowest average price** of any car company, their vehicles are the **least valuable**

### Model column

In [39]:
print("There are " + str(df['model'].nunique()) + " car models represented in the dataset.")
print('---'*25)
print("The unique car models are: " + str(df['model'].unique()))

There are 117 car models represented in the dataset.
---------------------------------------------------------------------------
The unique car models are: ['mpv' 'door' '1500' 'malibu' 'coupe' 'forte' 'encore' 'sorento' 'doors'
 'chassis' 'q70' 'camaro' 'convertible' 'vans' 'compass' 'enclave' '300'
 'cherokee' 'pacifica' 'x3' 'equinox' 'challenger' 'colorado' 'focus'
 'durango' 'escape' 'charger' 'explorer' 'f-150' 'caravan' 'van' 'dart'
 '2500' 'esv' 'el' 'edge' 'series' 'flex' 'srx' 'wagon' 'pickup' 'vehicl'
 'trax' 'tahoe' 'suburban' 'drw' 'fiesta' 'impala' 'cab' 'soul' 'elantra'
 'pioneer' 'trail' 'traverse' 'country' 'nautilus' 'q5' 'gle' 'sportage'
 '5' 'sport' 'discovery' 'acadia' 'ghibli' 'glc' 'e-class' 'cutaway'
 'utility' 'limited' 'cx-3' '2500hd' 'sonic' 'corvette' 'mdx' 'xt5'
 'fusion' 'mustang' '3500' 'passenger' 'volt' 'spark' 'cruze' 'ld'
 'journey' 'srw' 'transit' 'ranger' 'taurus' 'energi' 'expedition' 'max'
 'bus' 'ecosport' 'd' 'dr' 'hybrid' 'suv' 'connect' 'f-650

- There are 117 different car models in the dataset
- Let's check which model of car is most represented in the dataset for each company.

#### Popular Brand Model Combinations

In [40]:
brand_model_grouped = df.groupby(['brand', 'model'])['model'].count()

# Let's look at the first 10 rows of the new dataframe to see some popular brand model combinations
brand_model_grouped.reset_index(name='count').head(11)

,brand,model,count
0,acura,door,1
1,acura,mdx,1
2,audi,5,1
3,audi,q5,1
4,bmw,coupe,1
5,bmw,door,4
6,bmw,series,7
7,bmw,x3,2
8,buick,door,7
9,buick,enclave,2


**Observations**
- The 'series' model is the most popular BMW model
- The 'door' model is the most popular Buick model
- These model types are not very descriptive. BMW makes cars from series 2 up to series 8. We can only make reasonable guesses on which model car it is on the price

#### Let's look at the most popular brand model combinations in the dataset`

In [41]:
table = brand_model_grouped.sort_values(ascending=False).reset_index(name='count').head(10)

table

,brand,model,count
0,ford,door,272
1,ford,f-150,213
2,dodge,caravan,101
3,ford,doors,96
4,chevrolet,door,76
5,ford,fusion,65
6,dodge,durango,63
7,dodge,journey,61
8,nissan,rogue,54
9,dodge,door,52


In [42]:
fig = px.bar(table, x='brand', y='count', color='brand', text='model',
             title= 'Most Popular Brand Model combinations')

fig.update_xaxes(title='Brand Name', showgrid=False, showline=True, linewidth=2, linecolor='black')
fig.update_yaxes(title='Count of model', showgrid=False, showline=True)
fig.update_layout(font_size = 12, template='plotly_white', font_family='Arial')
fig.show()

- 8 of the 10 model combinations are vehicles made by ford and dodge which is to be expected since they make a majority of the listings.
- The most popular Ford vehicle is the 'Door model', followed by the F-150 
- The Caravan is the most popular Dodge vehicle, followed by the Durango

In [43]:
# Let's get a sense of the average price of each brand model combination in the dataset, and how many of each vehicle there are
brand_model_price = df.groupby(['brand', 'model'])['price'].agg({'mean', 'count'})

# Filter for cars that are represented at least 2 times
brand_model_price = brand_model_price[brand_model_price['count'] >=2]

# Top 5 most expensive brand model combinations
brand_model_price.sort_values(by='mean', ascending=False).head()

,,count,mean
brand,model,,
chevrolet,3500,2,"45,050.00"
ford,max,30,"41,980.00"
chevrolet,corvette,2,"41,450.00"
ford,expedition,20,"39,135.00"
chevrolet,tahoe,8,"37,862.50"


- The **Top 5 most expensive** brand model combinations are the **Chevrolet 3500**, the **Ford Max**, the **Chevrolet Corvette**, the **Ford Expedition**, and the **Chevrolet Tahoe**

In [44]:
brand_model_price.sort_values(by='mean').head()

,,count,mean
brand,model,,
hyundai,doors,4,"3,792.50"
chrysler,door,3,"4,073.33"
heartland,country,2,"4,450.00"
honda,doors,2,"4,660.00"
nissan,mpv,2,"4,755.00"


- The **Top 5 least expensive** brand model combinations are the **Hyundai Doors**, the **Chrysler door*, the **Heartland Country**, the **Honda Doors**, and the **Nissan MPV**.

### Color column

In [45]:
print("There are " + str(df['color'].nunique()) + " colors represented in the dataset.")
print('---'*25)
print("The unique car colors are: " + str(df['color'].unique()))

There are 44 colors represented in the dataset.
---------------------------------------------------------------------------
The unique car colors are: ['silver' 'blue' 'red' 'white' 'gray' 'black' 'orange' 'brown' 'no_color'
 'gold' 'charcoal' 'turquoise' 'beige' 'green' 'yellow' 'toreador red'
 'bright white clearcoat' 'billet silver metallic clearcoat'
 'black clearcoat' 'jazz blue pearlcoat' 'purple'
 'ruby red metallic tinted clearcoat' 'triple yellow tri-coat'
 'competition orange' 'color:' 'shadow black' 'magnetic metallic'
 'ingot silver metallic' 'ruby red'
 'royal crimson metallic tinted clearcoat' 'kona blue metallic'
 'oxford white' 'lightning blue' 'ingot silver'
 'white platinum tri-coat metallic' 'guard' 'tuxedo black metallic'
 'off-white' 'tan' 'super black' 'cayenne red' 'morningsky blue'
 'pearl white' 'glacier white']


In [46]:
# Top 10 most popular colors for a car
color_df = pd.DataFrame(df['color'].value_counts().head(10))

color_df.reset_index(inplace=True)

color_df.rename(columns={"index": 'color', "color": 'count'}, inplace=True)

color_df

,color,count
0,white,621
1,black,462
2,gray,365
3,silver,280
4,red,166
5,blue,127
6,no_color,48
7,charcoal,18
8,green,17
9,gold,14


In [47]:
fig = px.bar(color_df, y='count', x='color', template='plotly_white', title='Most Popular Car Colors', text='count')

fig.update_xaxes(title='Color of Vehicle', showline=True, linewidth=2, linecolor='black', ticks='outside')
fig.update_yaxes(title='Number of Vehicles', showgrid=True, showline=True, linewidth=1, linecolor='black')
fig.update_layout(font_size = 12, font_family='Arial')
fig.show()

- The most popular color is white, followed by black, gray and silver.
- There are 54 cars that do not have a color listed.
- There are 15 gold cars. Gold is usually associated with being more expensive and prestigious.

#### Potential analysis idea
- Average price of car by brand and color

In [48]:
# Average price of a car by it's color
color_price_df = df.groupby('color')['price'].agg({'mean', 'count'})

# Filter for colors which have at least 5 cars represented
color_price_df = color_price_df[color_price_df['count'] >=5]

# Top 5 most expensive colors
color_price_df.sort_values(by='mean', ascending=False).head()

,count,mean
color,,
shadow black,5,"26,260.80"
magnetic metallic,6,"23,838.17"
no_color,48,"21,263.62"
white,621,"19,882.22"
black,462,"19,838.26"


- The top 5 most expensive colors on average are shadow black, no_color, magnetic mettalic, black, and white

In [49]:
# Top 5 least expensive colors
color_price_df.sort_values(by='mean').head()

,count,mean
color,,
gold,14,"15,467.86"
orange,14,"16,285.00"
blue,127,"16,765.94"
charcoal,18,"16,822.22"
silver,280,"17,571.16"


- The top 5 least expensive colors are orange, charcoal, silver, gold, and gray

#### State column

In [50]:
print("There are " + str(df['state'].nunique()) + " states represented in the dataset.")
print('---'*25)
print("The states are: " + str(df['state'].unique()))

There are 42 states represented in the dataset.
---------------------------------------------------------------------------
The states are: ['georgia' 'virginia' 'florida' 'texas' 'california' 'tennessee'
 'north carolina' 'ohio' 'new york' 'pennsylvania' 'south carolina'
 'michigan' 'arizona' 'washington' 'kentucky' 'nebraska' 'ontario'
 'missouri' 'connecticut' 'arkansas' 'colorado' 'new jersey' 'wisconsin'
 'utah' 'illinois' 'oklahoma' 'minnesota' 'oregon' 'indiana'
 'west virginia' 'nevada' 'kansas' 'massachusetts' 'rhode island'
 'louisiana' 'alabama' 'mississippi' 'new mexico' 'idaho' 'new hampshire'
 'vermont' 'maryland']


In [51]:
state_df = pd.DataFrame(df['state'].value_counts().head(10))

state_df.reset_index(inplace=True)

state_df.rename(columns={"index": 'state', "state": 'count'}, inplace=True)

state_df

,state,count
0,pennsylvania,274
1,florida,221
2,california,175
3,texas,171
4,michigan,157
5,north carolina,144
6,minnesota,107
7,illinois,87
8,new jersey,85
9,wisconsin,85


In [52]:
fig = px.bar(state_df, y='count', x='state', template='plotly_white', title='States with most cars', text='count')

fig.update_xaxes(title='State', showline=True, linewidth=2, linecolor='black', ticks='outside')
fig.update_yaxes(title='Number of Vehicles', showgrid=True, showline=True, linewidth=1, linecolor='black')
fig.update_layout(font_size = 12, font_family='Arial')
fig.show()

- Pennsylvania is the most represented state
- Rounding out the top 5 are Florida, California, Texas, and Michigan.

#### Potential analysis idea
- Average price of car by state

In [53]:
# Average price of a car by state
state_price_df = df.groupby('state')['price'].agg({'mean', 'count'})

# Filter for states which have at least 5 cars represented
state_price_df = state_price_df[state_price_df['count'] >=5]

# Top 5 most expensive colors
state_price_df.sort_values(by='mean', ascending=False).head()

,count,mean
state,,
ontario,7,"30,357.14"
illinois,87,"24,673.56"
washington,10,"24,440.00"
michigan,157,"24,312.94"
mississippi,21,"23,747.62"


- The top 5 states/provinces with the most expensive cars are Kentucky, Ontario, Washington, Illinois and Indiana

In [54]:
# Top 5 least expensive states
state_price_df.sort_values(by='mean').head()

,count,mean
state,,
arkansas,7,"4,662.14"
texas,171,"13,277.85"
oregon,20,"14,385.00"
ohio,28,"14,731.79"
florida,221,"15,057.04"


- The top 5 least expensive states are Arkansas, Texas, Oregon, Connecticut and Arizona